# Astrogator Object Model Walkthrough in Python#

This tutorial builds a LEO to GEO transfer using a combined inclination and apogee raise maneuver at GEO.

## Uncomment and run the cell below the first time you connect to STK with Python on a new computer.

In [1]:
# # Import the comtypes package to connect to an external application
# import comtypes
# from comtypes.client import CreateObject

# # Create a new instance of STK
# app = CreateObject('STK11.Application')
# app.Visible = True # Needed to view the user interface application

# # Get the IAgStkObjectRoot interface
# root = app.Personality2

# # Note: When 'root=uiApplication.Personality2' is executed, the comtypes library automatically creates a gen folder that contains
# # STKObjects and other Python wrappers for the STK libraries. After running this at least once on your computer, the libraries
# # should be imported before the 'uiApplication=CreateObject("STK11.Application")' line for improved performance.

# # The Astrogator wrapper needs to specifically be loaded with the line below, then AgStkGatorLib is created.
# comtypes.client.GetModule((comtypes.GUID("{090D317C-31A7-4AF7-89CD-25FE18F4017C}") ,1,0))

# # View the location and newly generated wrappers
# import os as os
# print(comtypes.client.gen_dir, '\n')
# print(os.listdir(comtypes.client.gen_dir))

# # After running this cell comment out this cell.
# # Use "Ctrl" + "a" to select all of the cell content, then Use "Ctrl" + "/" to toggle comments in Jupyter Notebooks

## Look in the comments for the word ACTION, which indicates code is needed ##

## Connect to STK ##

In [2]:
# Import libraries
import comtypes
from comtypes.client import CreateObject
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.gen import AgStkGatorLib

In [3]:
# Create a new instance of STK, grab the root, create a new scenario
app = CreateObject("STK11.Application")
app.Visible = 1
app.UserControl = 1
root = app.Personality2
root.NewScenario("LEOToGEOWithPython");

In [4]:
# Grab scenario handles
sc = root.CurrentScenario
# ACTION: Get the scenario specific interface
sc2 = sc.QueryInterface(STKObjects.IAgScenario)

## Create an Astrogator Satellite ##

In [5]:
# Create a new satellite. Delete the satellite if it already exists.
satName = "AstrogatorSat"
if sc.Children.Contains(STKObjects.eSatellite, satName):
    sat = root.GetObjectFromPath("*/Satellite/" + satName)
    sat.Unload()

# ACTION: Insert a new satellite using the variable satName
sat = sc.Children.New(STKObjects.eSatellite, satName)

In [6]:
# Get a handle to the IAgSatellite interface
sat2 = sat.QueryInterface(STKObjects.IAgSatellite)
# Set the propagator to ePropagatorAstrogator
sat2.SetPropagatorType(STKObjects.ePropagatorAstrogator)
# Cast the propagator to an IAgVADriverMCS
ASTG = sat2.Propagator.QueryInterface(AgStkGatorLib.IAgVADriverMCS)

In [7]:
# Get a handle to the MCS
MCS = ASTG.MainSequence
type(MCS)

comtypes.POINTER(IAgVAMCSSegmentCollection)

## Build the MCS ## 

### Switching the "a" and "b" notion for different interfaces instead of "1" and "2" ###

In [8]:
# Add a stopping condition of descending node
prop1a = MCS.Item("Propagate")
prop1b = prop1a.QueryInterface(AgStkGatorLib.IAgVAMCSPropagate)
# ACTION: Add a stopping condition of descending node
prop1b.StoppingConditions.Add("DescendingNode");

In [9]:
# Add a target sequence
ts1a = MCS.Insert(AgStkGatorLib.eVASegmentTypeTargetSequence, "TargetApo", "-")
type(ts1a)

comtypes.POINTER(IAgVAMCSSegment)

In [10]:
# Get the interface specifically for the target sequence
ts1b = ts1a.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)

In [11]:
# Add the the first implusive maneuver
dV1a = ts1b.Segments.Insert(2, "dV", "-")
type(dV1a)

comtypes.POINTER(IAgVAMCSSegment)

In [12]:
# ACTION: Get the maneuver specific interface
dV1b = dV1a.QueryInterface(AgStkGatorLib.IAgVAMCSManeuver)

In [13]:
# Get the attitude control interface (direction of the maneuver)
dV1b.Maneuver.SetAttitudeControlType(AgStkGatorLib.eVAAttitudeControlThrustVector)
type(dV1b.Maneuver.AttitudeControl)

comtypes.POINTER(IAgVAAttitudeControl)

In [14]:
# Get specific the attitude control thrust vector inteface
attCon = dV1b.Maneuver.AttitudeControl.QueryInterface(
    AgStkGatorLib.IAgVAAttitudeControlImpulsiveThrustVector
)

In [15]:
# Set the direction and magnitude of the delta V vector
dVx = 1  # [km/s]
dVy = 0  # [km/s]
dVz = 0  # [km/s]
# ACTION: Assign the dV components to the delta V vector
attCon.DeltaVVector.AssignCartesian(dVx, dVy, dVz);

In [16]:
# Insert a propagate segment into the first target sequence
prop2a = ts1b.Segments.Insert(AgStkGatorLib.eVASegmentTypePropagate, "PropToApo", "-")
prop2b = prop2a.QueryInterface(AgStkGatorLib.IAgVAMCSPropagate)

In [17]:
color = 65535  # yellow. Decimal value for BGR
# ACTION: Change the segment color to use the color variable
prop2a.Properties.Color = color

In [18]:
# Set a stopping condition of apoapsis
prop2b.StoppingConditions.Add("AscendingNode");

In [19]:
# ACTION: Insert a second target sequence called TargetCircularization, after the first target sequence
ts2a = MCS.Insert(
    AgStkGatorLib.eVASegmentTypeTargetSequence, "TargetCircularization", "-"
)

In [20]:
# ACTION: Get the target sequence specific interface
ts2b = ts2a.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)

In [21]:
# Add a second impulsive maneuver
dV2a = ts2b.Segments.InsertCopy(dV1a, "-")
dV2b = dV2a.QueryInterface(AgStkGatorLib.IAgVAMCSManeuver)

In [22]:
# Insert a Propagate Segment
prop3a = ts2b.Segments.Insert(AgStkGatorLib.eVASegmentTypePropagate, "Prop1Day", "-")
prop3b = prop3a.QueryInterface(AgStkGatorLib.IAgVAMCSPropagate)

In [23]:
color = 16711935  # Magenta. Decimal value for BGR
# ACTION: Change the segment color to use the color variable
prop3a.Properties.Color = color

In [24]:
# Insert a stoppping condtion
scDura = prop3b.StoppingConditions.Item(0)
tripVal = 86400  # [sec], 86400 sec = 1 day
# ACTION: Add a stopping duration trip value using the tripVal variable (two lines of code)
scDurb = scDura.Properties.QueryInterface(AgStkGatorLib.IAgVAStoppingCondition)
scDurb.Trip = tripVal

## Configure the Target Sequences ## 

In [25]:
# Enable the control parameters
# ACTION: Enable the impulsive cartesian X component on the first dV maneuver
dV1b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianX)
# ACTION: Enable the impulsive cartesian X,Y, and Z components on the second dV maneuver (three lines of code)
dV2b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianX)
dV2b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianY)
dV2b.EnableControlParameter(AgStkGatorLib.eVAControlManeuverImpulsiveCartesianZ);

In [26]:
# Add results
prop2a.Results.Add("Spherical Elems\\R Mag")
dV2a.Results.Add("Keplerian Elems\\Eccentricity")
dV2a.Results.Add("Keplerian Elems\\Inclination")
dV2a.Results.Add("Spherical Elems\\Flight Path Angle");

In [27]:
# Enable the control parameters and results for the first differentical corrector
dc1a = ts1b.Profiles.Item(0)
dc1b = dc1a.QueryInterface(AgStkGatorLib.IAgVAProfileDifferentialCorrector)
cartXcp = dc1b.ControlParameters.Item(0)
cartXcp.Enable = 1
RMag = dc1b.Results.Item(0)
RMag.Enable = 1
# Set the desired radius to the GEO belt
RMag.DesiredValue = 42164  # [km]

In [28]:
# Enable the control parameters and results for the second differentical corrector
dc2a = ts2b.Profiles.Item(0)
dc2b = dc2a.QueryInterface(AgStkGatorLib.IAgVAProfileDifferentialCorrector)
# Alternative method of getting the control parameters
cartXcp = dc2b.ControlParameters.GetControlByPaths("dV", "ImpulsiveMnvr.Cartesian.X")
cartYcp = dc2b.ControlParameters.GetControlByPaths("dV", "ImpulsiveMnvr.Cartesian.Y")
cartZcp = dc2b.ControlParameters.GetControlByPaths("dV", "ImpulsiveMnvr.Cartesian.Z")
cartXcp.Enable = 1
cartYcp.Enable = 1
cartZcp.Enable = 1

In [29]:
# ACTION: Get the results by path and save the handles as variables called: ecc, inc, fpa (three lines of code)
ecc = dc2b.Results.GetResultByPaths("dV", "Eccentricity")
inc = dc2b.Results.GetResultByPaths("dV", "Inclination")
fpa = dc2b.Results.GetResultByPaths("dV", "Flight Path Angle")
ecc.Enable = 1
inc.Enable = 1
fpa.Enable = 1
# Adjust the toleraces
ecc.Tolerance = 0.001
inc.Tolerance = 0.001
fpa.Tolerance = 0.0001

In [30]:
# Set the Target Profiles Action to Run active profiles
ts1b.Action = AgStkGatorLib.eVATargetSeqActionRunActiveProfiles
ts2b.Action = AgStkGatorLib.eVATargetSeqActionRunActiveProfiles

## Run the MCS ##

In [31]:
# Optionally turn on and off the targetting pop up windows
DisplayStatus = 0  # 0 or 1, for don't show and show

for i in range(0, MCS.Count):
    seg = MCS.Item(i)
    if seg.Type == 8:
        ts = seg.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)
        dc = ts.Profiles.Item(0)
        dc = dc.QueryInterface(AgStkGatorLib.IAgVAProfileDifferentialCorrector)
        dc.EnableDisplayStatus = DisplayStatus;

In [32]:
# Run the MCS
ASTG.RunMCS();

In [33]:
# Apply the profile changes
ASTG.ApplyAllProfileChanges();

In [34]:
# Loop through every segment and set all target sequences to run nominal sequence
tsList = range(0, MCS.Count)
for i in range(0, MCS.Count):
    seg = MCS.Item(i)
    if seg.Type == 8:
        ts = seg.QueryInterface(AgStkGatorLib.IAgVAMCSTargetSequence)
        ts.Action = AgStkGatorLib.eVATargetSeqActionRunNominalSeq

In [35]:
# See what the options are for displaying orbits
sat2.VO.Pass.TrackData.PassData.Orbit.SupportedDataTypes

((1, 'All'),
 (8, 'Current Interval'),
 (2, 'Fraction'),
 (3, 'Full'),
 (4, 'Half'),
 (0, 'None'),
 (5, 'One Pass'),
 (6, 'Quarter'),
 (7, 'Time'))

In [36]:
# ACTION: Set the lead and trail type to All (two lines)
sat2.VO.Pass.TrackData.PassData.Orbit.SetLeadDataType(1)
sat2.VO.Pass.TrackData.PassData.Orbit.SetTrailDataType(1);

In [37]:
# Clear the graphics
ASTG.ClearDWCGraphics();